# Test 04 — BoardSDK (RGB / LED / Buzzer)

This test probes BoardSDK capabilities and exercises:
- `set_rgb(pixels: List[Tuple[idx,r,g,b]])`
- `set_led(...)`
- `set_buzzer(...)`

It tries safe call patterns and prints what worked.

In [ ]:
from pathlib import Path
import sys

def add_repo_root():
    here = Path.cwd().resolve()
    for p in [here] + list(here.parents):
        if (p / 'lessons').is_dir() and (p / 'common').is_dir():
            if str(p) not in sys.path:
                sys.path.insert(0, str(p))
            print('Repo root:', p)
            return p
    raise FileNotFoundError('Could not find repo root (needs lessons/ and common/)')

add_repo_root()

In [ ]:
import time, inspect
from fast_sdk.board_sdk import BoardSDK

board = BoardSDK()
print('BoardSDK:', board)

cands = [n for n in dir(board) if any(k in n.lower() for k in ('buzzer','rgb','led'))]
print('Candidates:', cands)

def sig(name):
    if not hasattr(board, name):
        return None
    try:
        return str(inspect.signature(getattr(board, name)))
    except Exception:
        return '<no signature>'

print('set_rgb signature:', sig('set_rgb'))
print('set_led signature:', sig('set_led'))
print('set_buzzer signature:', sig('set_buzzer'))

## RGB test

Your image reports:
`set_rgb(pixels: List[Tuple[int,int,int,int]])`

So we send e.g. `[(0,r,g,b),(1,r,g,b)]`.

In [ ]:
def set_eyes(r, g, b, pixels=(0, 1)):
    if not hasattr(board, 'set_rgb'):
        raise RuntimeError('BoardSDK has no set_rgb')
    payload = [(int(i), int(r), int(g), int(b)) for i in pixels]
    board.set_rgb(payload)

print('RGB cycle: red, green, blue, off')
set_eyes(255, 0, 0); time.sleep(0.35)
set_eyes(0, 255, 0); time.sleep(0.35)
set_eyes(0, 0, 255); time.sleep(0.35)
set_eyes(0, 0, 0);   time.sleep(0.2)
print('RGB OK')

## LED test

We try a few common call shapes for `set_led`.
If one works, we blink it a couple of times.

In [ ]:
def try_call(fn, *args):
    try:
        fn(*args)
        return True, None
    except Exception as e:
        return False, str(e)

def led_onoff(on: bool):
    if not hasattr(board, 'set_led'):
        return False, 'no set_led'
    fn = board.set_led
    patterns = [
        (1 if on else 0,),
        (bool(on),),
        ([1 if on else 0],),
        ([(1 if on else 0)],),
    ]
    for args in patterns:
        ok, err = try_call(fn, *args)
        if ok:
            return True, f'set_led{args}'
    return False, f'no matching signature for set_led (last err: {err})'

ok, how = led_onoff(True)
print('LED ON:', ok, how)
time.sleep(0.25)
ok, how = led_onoff(False)
print('LED OFF:', ok, how)

if ok:
    for _ in range(2):
        led_onoff(True); time.sleep(0.2)
        led_onoff(False); time.sleep(0.2)
    print('LED blink done')
else:
    print('LED not controllable with tested patterns')

## Buzzer test

We try safe patterns:
- `set_buzzer(1/0)`
- `set_buzzer(True/False)`
- `set_buzzer(freq)`
- `set_buzzer(freq, duration)`

If none match, it prints the signature.

In [ ]:
def buzzer(on=True, freq=2000, dur=0.2):
    if not hasattr(board, 'set_buzzer'):
        return False, 'no set_buzzer'
    fn = board.set_buzzer

    patterns = [
        (1 if on else 0,),
        (bool(on),),
        (int(freq),),
        (int(freq), float(dur)),
        (bool(on), int(freq)),
        (bool(on), int(freq), float(dur)),
    ]

    last = None
    for args in patterns:
        ok, err = try_call(fn, *args)
        if ok:
            return True, f'set_buzzer{args}'
        last = err
    return False, f'no matching signature (last err: {last}); sig={sig("set_buzzer")}'

ok, how = buzzer(True)
print('Buzzer ON:', ok, how)
time.sleep(0.25)
ok, how = buzzer(False)
print('Buzzer OFF:', ok, how)

if not ok:
    print('If buzzer should exist, paste set_buzzer signature + errors and we’ll lock it down.')